# Function Calling and Agents: Calculator

In [1]:
import ollama

## Function Calling

### Define the Tool (Function Schema)

In [4]:
multiply_tool = {
    "type": "function",
    "function": {
        "name": "multiply_numbers",
        "description": "Multiply two numbers together",
        "parameters": {
            "type": "object",
            "required": ["a", "b"],
            "properties": {
                "a": {"type": "number", "description": "First number"},
                "b": {"type": "number", "description": "Second number"}
            }
        }
    }
}

### Implement the Function (with type conversion)

In [3]:
def multiply_numbers(a, b):
    # Convert to int or float as needed
    a = float(a)
    b = float(b)
    return {"result": a * b}

### Orchestrate with Ollama (Synchronous Version)

In [21]:
def answer_query(QUERY):
    # User asks to multiply 123456 x 123456
    response = ollama.chat(
        'llama3.2:3B',
        messages=[{"role": "user", "content": QUERY}],
        tools=[multiply_tool]
    )

    # Check if the model wants to call the tool
    if response.message.tool_calls:
        for tool in response.message.tool_calls:
            if tool.function.name == "multiply_numbers":
                # Ensure arguments are passed as numbers
                result = multiply_numbers(**tool.function.arguments)
                print(f"Result: {result['result']:,.2f}")
            else:
                print(f"It is not a Multiplication")
            

In [22]:
QUERY = "What is 123456 x 123456?"

In [23]:
answer_query(QUERY)

Result: 15,241,383,936.00


In [25]:
answer_query("What is the capital of Brazil?")

Result: 0.00


`==> The above approach only works properly for the specific tool`

## Ollama-Agent with Python

Run on terminal: func_calling_.py

## Agents

- `pip install smolagents`
- `pip install "smolagents[toolkit]"`
- `pip install 'smolagents[litellm]'` 

In [46]:
from smolagents import CodeAgent, LiteLLMModel, tool, PromptTemplates

### Step 1: Define the tool function with a proper docstring

In [29]:
@tool
def multiply_calc(a: float, b: float) -> float:
    """Returns the product of two numbers.
    
    Args:
        a: The first number to multiply.
        b: The second number to multiply.
        
    Returns:
        float: The product of a and b.
    """
    return a * b

### Step 2: Create the agent

In [43]:
# Initialize model
model = LiteLLMModel(
    model_id="ollama/llama3.2:3B",
    api_base="http://localhost:11434",
    api_key="ollama",
    temperature=0.3,
    flatten_messages_as_text=True
)

In [51]:
agent = CodeAgent(
    tools=[multiply_calc],
    model=model,
    executor_type="local",
    max_steps=10
)

### Run the agent

In [52]:
response = agent.run("What is 123456 x 123456?")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is 123456 x 123456?                                                                                        │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/llama3.2:3B ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = multiply_calc(a=123456, b=123456)                                                                       
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 15241383936

[Step 1: Duration 277.21 seconds| Input tokens: 2,058 | Output tokens: 61]

15241383936


`==> The above approach only works properly for the specific tool`

## Enhanced Agent

### Creating a new Tool

In [54]:
@tool
def general_knowledge_query(query: str) -> str:
    """Answers general knowledge questions.
    
    This tool allows the agent to answer questions about facts, geography,
    history, and other general knowledge topics.
    
    Args:
        query: The question to answer.
        
    Returns:
        str: The answer to the question.
    """
    # This is a mock implementation. In a real scenario, you would
    # implement this with a knowledge base lookup or API call.
    # For demonstration purposes, we'll hardcode some answers:
    knowledge_base = {
        "capital of colombia": "Bogotá",
        "capital of france": "Paris",
        "capital of japan": "Tokyo",
        "capital of australia": "Canberra",
        "capital of brazil": "Brasília",
        # Add more knowledge as needed
    }
    
    # Convert query to lowercase for case-insensitive matching
    query_lower = query.lower()
    
    # Check for direct matches or partial matches
    for key, value in knowledge_base.items():
        if key in query_lower or query_lower in key:
            return value
    
    # If no match is found in our simple knowledge base
    return "I don't have information about that. Please try a different question or use a more advanced model with broader knowledge."


### Creating the agent with both tools

In [55]:
agent = CodeAgent(
    tools=[multiply_calc, general_knowledge_query],
    model=LiteLLMModel(
        model_id="ollama/llama3.2:3B", 
        api_base="http://localhost:11434",
        api_key="ollama",
        temperature=0.3,
        num_ctx=4096
    ),
    executor_type="local",
    max_steps=10
)

In [56]:
def run_query(query):
    response = agent.run(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print("-" * 50)

In [57]:
# Test multiplication
run_query("What is 25 multiplied by 17?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is 25 multiplied by 17?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/llama3.2:3B ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = multiply_calc(a=25, b=17)                                                                               
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 425

[Step 1: Duration 279.33 seconds| Input tokens: 2,112 | Output tokens: 46]

Query: What is 25 multiplied by 17?
Response: 425
--------------------------------------------------


In [58]:
# Test general knowledge
run_query("What is the capital of Colombia?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of Colombia?                                                                                │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/llama3.2:3B ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  capital = general_knowledge_query(query="Capital of Colombia")                                                   
  final_answer(capital)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Bogotá

[Step 1: Duration 17.63 seconds| Input tokens: 2,110 | Output tokens: 60]

Query: What is the capital of Colombia?
Response: Bogotá
--------------------------------------------------


In [59]:
# Test multiplication
run_query("What is 123456 multiplied by 123456?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is 123456 multiplied by 123456?                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/llama3.2:3B ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = multiply_calc(a=123456, b=123456)                                                                       
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
15241383936

Out: None

[Step 1: Duration 12.91 seconds| Input tokens: 2,114 | Output tokens: 43]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = multiply_calc(a=123456, b=123456)                                                                       
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 15241383936

[Step 2: Duration 29.06 seconds| Input tokens: 4,349 | Output tokens: 87]

Query: What is 123456 multiplied by 123456?
Response: 15241383936
--------------------------------------------------
